# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [2]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

This line may take as long as 10-20 minutes depending on your network connection and computer specs

Now it's time to explore the data!

In [4]:
import psycopg2
import pandas as pd

In [5]:
DBNAME = "opportunity_youth"

In [6]:
conn = psycopg2.connect(dbname=DBNAME, user="postgres", password="lovre2002")

In [7]:
pd.set_option('max_colwidth',0)

In [8]:
query = """
SELECT * 
FROM puma_names_2010 
WHERE puma_name like 'King County (South%'
LIMIT 10;
"""
pd.read_sql(query, conn)

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,53,Washington,1044,11613,5311613,G53011613,King County (Southwest Central)--Kent City
1,53,Washington,1044,11614,5311614,G53011614,King County (Southwest)--Auburn City & Lakeland
2,53,Washington,1044,11615,5311615,G53011615,"King County (Southeast)--Maple Valley, Covington & Enumclaw Cities"


In [9]:
pd.read_sql("""select count(serialno) from pums_2017 group by puma LIMIT 50""", conn)

,count
0,9168
1,18484
2,5342
3,8233
4,5958
5,6386
6,5806
7,5416
8,8340
9,5022


In [12]:
query = """
SELECT puma, agep, count(sporder) 
FROM pums_2017
WHERE (puma = '11613' OR puma = '11614' OR puma = '11615')
AND (agep = '16' 
OR agep ='17' 
OR agep = '18' 
OR agep = '19' 
OR agep = '20' 
OR agep = '21'
OR agep = '22'
OR agep = '23'
OR agep = '24')
AND cow = '9' 
AND fschp = '0'
GROUP BY puma, agep;
"""
pd.read_sql(query, conn)

,puma,agep,count
0,11613,16.0,3
1,11613,17.0,3
2,11613,20.0,2
3,11613,22.0,2
4,11614,16.0,4
5,11614,17.0,2
6,11614,18.0,2
7,11614,19.0,3
8,11614,22.0,3
9,11614,24.0,1


In [ ]:
query = """
SELECT count(serialno) 
FROM pums_2017
"""
pd.read_sql(query, conn)

puma = region (11613 - 11615 for S. King County)
agep = age (16-24)
cow = employment (9 = unemployed)
fschp = school enrollment (1 = yes, 0 = no)

Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [ ]:
query = """
SELECT * 
FROM ct_puma_xwalk 
LIMIT 10;
"""
pd.read_sql(query, conn)

Make sure you close the DB connection when you are done using it

In [ ]:
conn.close()